In [1]:
import findspark
import time
# mlibrerias modelos score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import PolynomialFeatures
from tpot.builtins import StackingEstimator, ZeroCount
findspark.init('/usr/hdp/2.5.0.0-1245/spark2')

In [2]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
from pyspark.sql import HiveContext
hive_context = HiveContext(sc)
import pandas as pd
from pyspark.sql.functions import *
import pickle

In [3]:
##hive_context.sql("show databases").show()


In [25]:
all_dataf_1 = hive_context.sql("select * from dev.rafael_traf_fcdr_2d")
all_datam_1 = hive_context.sql("select * from dev.rafael_traf_mcdr_2d")
all_datam = all_datam_1.toPandas()
all_dataf = all_dataf_1.toPandas()
#all_datam.head()

In [5]:
new_columns_m = ['rec_type',
                   'called_number',
                   'called_imei',
                   'called_imsi',
                   'cllng_number',
                   'cllng_imei',
                   'cllng_imsi',
                   'call_duration',
                   'cause_for_term',
                   'exchange_id',
                   'fcalled_ci',
                   'fcllng_ci',
                   'in_asig_route',
                   'in_route', 
                   'lcalled_ci',
                   'lcllng_ci',
                   'org_called_num',
                   'out_route',
                   'int_rec_number',
                   'int_charg_ind',
                   'rdirect_imsi',
                   'rdirect_number',
                   'start_time', 
                   'end_time',
                   'basic_serv_type',
                   'facility_usage',
                   'camel_call_ref',
                   'camel_exch_id',
                   'cllng_first_lac',
                   'clled_first_lac',
                   'emer_call_cat',
                   'clln_number_ton',
                   'dial_number_ton',
                   'cellid_lac_a',
                   'cellid_lac_b',
                   'cdr_date', 
                   'pais',
                   '_c2',
                   '_c3']
new_columns_f = ['abonado_origen',
'called_number',
'cllng_number',
'abonado_destino_transformado',
'abonado_destino_original',
'fecha',
'hora_inicio',
'call_duration',
'tipo_cdr',
'ruta_entrante',
'ruta_saliente',
'x_alias_central',
'cod_localidad_origen',
'cod_localidad_destino',
'cod_municipio_origen',
'cod_municipio_destino',
'cod_area_geografica_origen',
'cod_area_geografica_destino',
'cod_pais_origen',
'cod_pais_destino',
'portabilidad_numerica',
'fecha_migracion',
'start_time',
'fecha_fin',
'cdr_date',
'pais',
'_c2',
'_c3']

In [6]:
all_dataf.columns = new_columns_f
all_datam.columns = new_columns_m



columns_selection = ['cllng_number','called_number','start_time',
                     'call_duration','cdr_date']

all_datam_final = all_datam[columns_selection]
all_dataf_final = all_dataf[columns_selection]

all_datam_final.dtypes
all_dataf_final.dtypes
#ll_ldi_fraud_input.dtypes
all_ldi_fraud_input = pd.concat([all_datam_final,all_dataf_final], axis=0, ignore_index=True)
all_data=all_ldi_fraud_input

In [7]:
def get_attrib_fraud_score(all_data):
    import pandas as pd
    import datetime   
    output_final=pd.DataFrame()
    
    all_data['call_duration']=pd.to_numeric(all_data['call_duration'])
    all_data.cdr_date=pd.to_datetime(all_data.cdr_date)
    all_data['start_time_1']= all_data['start_time'].astype(str)
    all_data['start_time_1']=pd.to_datetime(all_data['start_time_1'],format='%Y%m%d%H%M%S',errors ='coerce' )
    
    
    #table_attributes['tot_time']=all_data.groupby(['cllng_number'])['call_duration'].sum()
    #table_attributes['avg_time']=all_data.groupby(['cllng_number'])['call_duration'].mean()
    #table_attributes['T_unique_numbers']=all_data.groupby(['cllng_number']).called_number.nunique()
    #table_attributes['tot_calls']=all_data.groupby(['cllng_number'])['cllng_number'].count()
    #table_attributes['stdev_duration_calls']=all_data.groupby(['cllng_number'])['call_duration'].std()
    #table_attributes['scattered_calls']=all_data.groupby(['cllng_number','call_type'])['start_time_1'].std()

    """  last day attributes creation must be calcultated for each number  with the 
    following guidelines:  took  last record time (start_time),  then develop
    all attributes 24 hours before this time
    """
    
    """ last day attributes creation must be calcultated for each number  with the 
    following guidelines:  took  last record time (start_time),  then develop
    all attributes 24 hours before this time
    """
    bad_attributes=pd.DataFrame()
    
    bad_attributes['cllng_number']=all_data['cllng_number'].unique()
    #test_0=last_24hr_cdr['cllng_number'].unique()
    #output_final=pd.DataFrame()



    ## funcion para crear columnas planas 
    def flatten_cols(df):
        df.columns = [
                '_'.join(tuple(map(str, t))).rstrip('_') 
                for t in df.columns.values
                ]
        return df
    ## end of function 
        
   # end function Hour of the day
    
    def hr_day_func(ts):
        hora_dia= ts.hour
        return hora_dia  
    
    # end function to get day hour

   # lista =  bad_attributes['cllng_number'].unique()
    

    
    #  lista = telecom_List
     #k=lista[0]
    #import numpy as np
   
    #all_data['cllng_number'] = pd.to_numeric(all_data['cllng_number'])
    #all_data['cllng_number']= all_data['cllng_number'].astype(np.int64)
	

        #     TELEFONO	OPERADOR	FECHA_DETECCION
        #    15673069	TELECOM	2018-05-06
        # 3187544004,TELECOM,1/30/2018

    
        
        
        #celular=all_data[all_data['cllng_number'] == 15688540]
        #celular  = celular[ celular['cdr_date'] <= '2018-06-11'] 
        # num_unicos['numero']                            
        
        #
        
    weeks_before_2 = all_data['cdr_date'].max()- datetime.timedelta(days=15)
    ## ya se  realiza la extraccion paa las ultimas dos horas
    #hours_before_2 = row['FECHA_DETECCION']- datetime.timedelta(hours=3)
    # celular dataframe with info from last 2 weeks
    #celular  = celular[ celular['start_time_1'] >= weeks_before_2]
    
    yesterday = all_data.cdr_date.max() - datetime.timedelta(days=1)  
    
    # dataframe with cdr data from their last day 
    last_2day_cdr=all_data.query('cdr_date >= @yesterday' ) 
    
    
    table_attributes=pd.DataFrame()
    table_attributes['tot_time']=all_data.groupby(['cllng_number'])['call_duration'].sum()
    table_attributes['avg_time']=all_data.groupby(['cllng_number'])['call_duration'].mean()
    table_attributes['T_unique']=all_data.groupby(['cllng_number'])['called_number'].nunique()
    table_attributes['tot_cal']=all_data.groupby(['cllng_number'])['cllng_number'].count()
    table_attributes['stdev_dur_cal']=all_data.groupby(['cllng_number'])['call_duration'].std()
    
    
    #table_attributes=pd.DataFrame()
    # get calculated from total last day     
    table_attributes['tot_time_2d']=last_2day_cdr.groupby(['cllng_number'])['call_duration'].sum()
    table_attributes['avg_time_2d']=last_2day_cdr.groupby(['cllng_number'])['call_duration'].mean()
    table_attributes['T_unique_2d']=last_2day_cdr.groupby(['cllng_number'])['called_number'].nunique()
    table_attributes['tot_cal_2d']=last_2day_cdr.groupby(['cllng_number'])['cllng_number'].count()
    table_attributes['stdev_dur_cal_2ld']=last_2day_cdr.groupby(['cllng_number'])['call_duration'].std()
    #   table_attributes['scattered_cal_ld']=last_day_cdr.groupby(['cllng_number','call_type'])['start_time_1'].std()
    
    # get attributes from last 24 hours
    
    T_24hr = all_data.start_time_1.max() - datetime.timedelta(hours = 24)  

    
    last_24hr_cdr=all_data.query('start_time_1 >= @T_24hr' ) 
    # function for get hour delta 
    def hr_func(ts):
        if last_24hr_cdr.start_time_1.max().date() == ts.date():
            return  last_24hr_cdr.start_time_1.max().hour-ts.hour
        else:    return  last_24hr_cdr.start_time_1.max().hour + (24-ts.hour)
        
     
    def quarter_time_func(ts):
        return  (celular_hour.start_time_1.max()-ts).seconds/60
        
        
          
    
    def hr_group_func(ts):
        if ts <= 4: return 4
        elif ts>4 and ts<= 8: return 8
        elif ts>8 and ts<=12: return 12
        elif ts>12 and ts <=16 : return 16
        elif ts>16 and ts<=20: return 20 
        else :
            return 24  
    
    
    
    # end function Hour of the day
    # rafael 23 de octubre de 2018
    pd.options.mode.chained_assignment = None  # default='warn
    last_24hr_cdr['hora_0']= last_24hr_cdr['start_time_1'].apply(hr_func)
    last_24hr_cdr['hora']= last_24hr_cdr['hora_0'].apply(hr_group_func)
    last_24hr_cdr['hora_dia']= last_24hr_cdr['start_time_1'].apply(hr_day_func)
    attributes_hora_dia=pd.DataFrame()
    attributes_hora_dia['sum_call_time_Hdia_out']=last_24hr_cdr.groupby(['cllng_number','hora_dia'])['call_duration'].sum()
    attributes_hora_dia['tot_call_Hdia_out']=last_24hr_cdr.groupby(['cllng_number','hora_dia'])['call_duration'].count()

    
    
    
    # test=attributes_day.unstack(level=1)
    #test_1=test.unstack(level=1)
    # data fram with tot call and sum of call time by day hour for  last 24 hours
    t_attr_dia=attributes_hora_dia.unstack(level=1)
    
    output_celular=pd.DataFrame()   
    #flatten_cols(test)
    flatten_cols(t_attr_dia)
        
    """ 
    Section to get attributes last 2 hours

    """
    test_quarter = pd.DataFrame()
    import tqdm


    num_unicos =pd.DataFrame()
    num_unicos['numero']= last_24hr_cdr['cllng_number'].unique()
    
    #num_unicos['numero']=pd.to_numeric(num_unicos['numero'])
    #all_data.dtypeslast_24hr_cdr
    
    for index,row in num_unicos.iterrows():
        #print(index/17081)
        celular=pd.DataFrame()
        
        time.sleep(0.01)
        celular=last_24hr_cdr[last_24hr_cdr['cllng_number'] == row['numero']]  
        #celular=last_24hr_cdr[last_24hr_cdr['cllng_number'] == '3168456745']
        
        #celular  = celular[ celular['cdr_date'] <= row['FECHA_DETECCION']]
        celular['call_duration']=pd.to_numeric(celular['call_duration'])
        celular.cdr_date=pd.to_datetime(celular.cdr_date)
        celular.cdr_date.max()
        
        celular_ult_horas = pd.DataFrame()
        time_ventana=celular['start_time_1'].max() - datetime.timedelta(hours=2)
        celular_ult_horas =celular[ celular['start_time_1'] >= time_ventana ] 
                           
        #print(row['numero'],len(celular))
        
        if len(celular_ult_horas)==0: 
            continue 
        celular_ult_horas['call_duration']=pd.to_numeric(celular_ult_horas['call_duration'])
        celular_ult_horas.cdr_date=pd.to_datetime(celular_ult_horas.cdr_date)

        celular_ult_horas['call_duration']=pd.to_numeric(celular_ult_horas['call_duration'])
        celular_ult_horas.cdr_date=pd.to_datetime(celular_ult_horas.cdr_date)
        celular_ult_horas['start_time_1']= celular_ult_horas['start_time'].astype(str)
        celular_ult_horas['start_time_1']=pd.to_datetime(celular_ult_horas['start_time_1'],format='%Y%m%d%H%M%S')
        
        #agg_10m = df.groupby(pd.Grouper(freq='10Min')).aggregate(numpy.sum)

        #celular_ult_horas['agg_10m'] = celular_ult_horas.groupby(pd.Grouper(key= '', freq='15Min'))
        ult_2_horas = pd.DataFrame()
        ult_2_horas['15min']=  celular_ult_horas.groupby(['cllng_number', pd.Grouper(key='start_time_1', freq='15Min')])['call_duration'].sum()
        ult_2_horas.reset_index(level=0, inplace=True)
        ult_2_horas['time'] = ult_2_horas.index
        #ult_2_horas.query('cllng_number==3005192893')
        
        #def flatten_cols(df):
        #    df.columns = [
        #        '_'.join(tuple(map(str, t))).rstrip('_') 
        #        for t in df.columns.values
        #        ]
        #return df
    
            
       #test.start_time.max().minuter-ts.minutehour    
    
        def quarter_time_func(ts):
            return  (celular.start_time_1.max()-ts).seconds/60
    
        # funcion para calcular los ultimas horas en promedios de 15 minutos
    
        def hr_group_func(ts):
            if ts <= 15: return 15
            elif ts>15 and ts<= 30: return 30
            elif ts>30 and ts<=45: return 45
            elif ts>45 and ts <=60 : return 60
            elif ts>60 and ts<=75: return 75 
            elif ts>75 and ts<=90: return 90
            elif ts>90 and ts<=105: return 105
            else :
                return 120
            
            
        celular_ult_horas['minutes']=  celular_ult_horas['start_time_1'].apply(quarter_time_func)   
        celular_ult_horas['quarter']=  celular_ult_horas['minutes'].apply(hr_group_func)
     
     
        table_attributes_ult_horas=pd.DataFrame()
        # get calculated from total last day     
        table_attributes_ult_horas['tot_time']=celular_ult_horas.groupby(['cllng_number','quarter'])['call_duration'].sum()
        table_attributes_ult_horas['avg_time']=celular_ult_horas.groupby(['cllng_number','quarter'])['call_duration'].mean()
        table_attributes_ult_horas['T_unique']=celular_ult_horas.groupby(['cllng_number','quarter'])['called_number'].nunique()
        table_attributes_ult_horas['tot_cal']=celular_ult_horas.groupby(['cllng_number','quarter'])['cllng_number'].count()
        # table_attributes_ult_horas['stdev_dur_cal_ld']=celular_ult_horas.groupby(['cllng_number'])['call_duration'].std()

        test_quarter_part=table_attributes_ult_horas.unstack(level=1)
        flatten_cols(test_quarter_part)
        
        test_quarter= pd.concat([test_quarter,test_quarter_part], axis=0, sort = True )
        
    """ 
    end Section to get attributes last 2 hours
    see  results on test_quarter table
    """
        
        #table_attributes.columns
        # test_1.columns = test_1.columns.tolist()
        
        # case empty table attributes 3196422755
        # case   several days 3002161926
        
        # start if
        
        #if len(table_attributes) == 0:  
        #    s=pd.DataFrame(columns=['tot_time_ld', 'avg_time_ld', 'T_unique_ld', 'tot_cal_ld',
        #                            'stdev_dur_cal_ld'], index=[str(row['TELEFONO'])])
        #    table_attributes=s
        #    #table_attributes=table_attributes.append(s, ignore_index = False)
        #    output_celular=pd.concat([table_attributes.reset_index(drop=True), test.reset_index(drop=True)], axis=1)
        #    # change the index value 
        #    as_list = output_celular.index.tolist()
        #    idx = as_list.index(0)
        #    as_list[idx] = row['TELEFONO']
        #    output_celular.index = as_list
        #    output_celular['Call_num'] = output_celular.index
        #else:
            # table_attributes_1=table_attributes.unstack(level=2)
            #flatten_cols(table_attributes)
    output_celular=pd.merge(table_attributes,t_attr_dia, right_index=True, left_index=True, sort = True)
    output_celular=pd.merge(output_celular,test_quarter, right_index=True, left_index=True,how = "left", sort = True)
    #output_celular=pd.merge(output_celular,table_attributes_ult_horas, right_index=True, left_index=True)
    #table_attributes_ult_horas    #output_celular=pd.merge(output_celular,table_attributes_ult_horas, right_index=True, left_index=True)
            
            
    output_celular['Call_num'] = output_celular.index
            # output_final=output_final.append(output_celular, sort =True)
               
        #if k==lista[0]:
        #    print(k)
        #    output_final=output_celular
        #else:
        #print(k)
            #output_final_0 = pd.concat(list(output_final.align(output_celular)), ignore_index=True)
    #output_final = pd.concat([output_final,output_celular], axis=0, ignore_index=True)

        
       
    del table_attributes ,test_quarter   


    #output_final=output_final.join(output_celular,on='Call_num',
    #                                    lsuffix='_left', rsuffix='_right')
    

    return output_celular


In [8]:

resultados = get_attrib_fraud_score(all_ldi_fraud_input)

In [24]:
#resultados.head()

In [22]:
#output_celular.head()

In [23]:
#all_datam.head()

In [12]:
#cols_remove=['tot_time_2d'
#,'avg_time_2d'
#,'T_unique_2d'
#,'tot_cal_2d'
#,'stdev_dur_cal_2ld'
#]

In [13]:
#features_testing = resultados.drop(cols_remove, axis=1)

In [14]:
#exported_pipeline = make_pipeline(
#    StackingEstimator(estimator=BernoulliNB(alpha=1.0, fit_prior=True)),
#    PolynomialFeatures(degree=2, include_bias=False, interaction_only=False),
#    ZeroCount(),
#    RandomForestClassifier(bootstrap=True, criterion="entropy",
#                           max_features=0.4, min_samples_leaf=12,
#                           min_samples_split=3, n_estimators=100)
#)

In [13]:
# Load Model
# load the model from local
ruta = '/home/dataloaderusr/fraude_model_19mar.sav'
loaded_model = pickle.load(open(ruta, 'rb'))
#result = loaded_model.score(X_test, Y_test)
#print(result)

In [20]:
# scoring
#ruta_score = 'c:/data/trafico_ldi/scoring_input/score_input_dic_1.csv'
#input_score = pd.read_csv(ruta_score , sep=',', dtype=np.float64)
cols_remove=['tot_time_2d'
,'avg_time_2d'
,'T_unique_2d'
,'tot_cal_2d'
,'stdev_dur_cal_2ld'
, 'Call_num'
             
]
resultados_1 = resultados.fillna(0) 
features_testing =  resultados_1.drop(cols_remove, axis=1)


result_score = loaded_model.predict_proba(features_testing)
#sum(result_score)
#import pandas as pd
salida_marcacion = pd.DataFrame(result_score)

In [26]:
# output fraude score 
salida_marcacion.to_csv('/home/dataloaderusr/output_fraude.csv')